In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.cluster import DBSCAN
from matplotlib import cm
from sklearn.ensemble import IsolationForest
import numpy as np
import xgboost as xgb
import seaborn as sns
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.covariance import EllipticEnvelope
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics


In [2]:
X = pd.read_csv('Dorothea/dorothea_train.data', delimiter=' ', names=list(range(6060)))
y = pd.read_csv('Dorothea/dorothea_train.labels', names=list(range(1)))

for i in range(len(X.columns)):
    X[i].fillna(X[i].median(), inplace=True)

X.head()

,0,1,2,3,4,5,6,7,8,9,...,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059
0,191,367,614,634,711,1202,1220,1311,1472,1730,...,99821.0,99838.0,99851.0,99862.0,99889.0,99896.0,99924.0,99930.0,99934.0,99976.0
1,118,307,367,478,505,512,807,878,939,1024,...,99821.0,99838.0,99851.0,99862.0,99889.0,99896.0,99924.0,99930.0,99934.0,99976.0
2,10,184,284,297,320,375,445,588,658,1108,...,99821.0,99838.0,99851.0,99862.0,99889.0,99896.0,99924.0,99930.0,99934.0,99976.0
3,87,149,433,704,711,892,988,1056,1070,1234,...,99821.0,99838.0,99851.0,99862.0,99889.0,99896.0,99924.0,99930.0,99934.0,99976.0
4,84,118,279,316,435,505,584,629,849,1029,...,99821.0,99838.0,99851.0,99862.0,99889.0,99896.0,99924.0,99930.0,99934.0,99976.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
svm2_clf = svm.SVC(kernel='linear')

svm2_clf.fit(X_train, y_train)
expected = y_test
predicted = svm2_clf.predict(X_test)
print(metrics.classification_report(expected, predicted))

/home/e/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

          -1       0.90      0.91      0.90       216
           1       0.09      0.08      0.09        24

    accuracy                           0.82       240
   macro avg       0.49      0.50      0.50       240
weighted avg       0.82      0.82      0.82       240



In [4]:
df=pd.concat((X,y),axis=1)
df2=df

In [5]:
#Outlier Detection process
scaler = MinMaxScaler()
num = scaler.fit_transform(df)
df = pd.DataFrame(num, columns = df.columns)
print(df.shape)
df.head()

(800, 6061)


,0,1,2,3,4,5,6,7,8,9,...,6051,6052,6053,6054,6055,6056,6057,6058,6059,0
0,0.215909,0.351562,0.475240,0.378328,0.383815,0.629164,0.497818,0.499591,0.544735,0.466992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.132955,0.292969,0.277955,0.281734,0.264740,0.252321,0.317627,0.322567,0.333729,0.264351,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.010227,0.172852,0.211661,0.169659,0.157803,0.177499,0.159686,0.204007,0.222486,0.288462,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.097727,0.138672,0.330671,0.421672,0.383815,0.459858,0.396597,0.395339,0.385590,0.324627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.094318,0.108398,0.207668,0.181424,0.224277,0.248498,0.220332,0.220769,0.298100,0.265786,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Isolation Forest
rs=np.random.RandomState(0)
clf = IsolationForest(max_samples=100,random_state=rs, contamination=.1) 
clf.fit(df)
if_scores = clf.decision_function(df)
if_anomalies=clf.predict(df)
if_anomalies1=pd.Series(if_anomalies).replace([-1,1],[1,0])


/home/e/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/e/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [8]:
TT=df.loc[if_anomalies1==1]
df=df.drop(TT.T)
X=df.iloc[:, :-1].values
y=df.iloc[:, 6060].values

In [11]:
y.shape

(720,)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [10]:
svm2_clf = svm.SVC(kernel='linear')
from sklearn import svm, metrics
svm2_clf.fit(X_train, y_train)
expected = y_test
predicted = svm2_clf.predict(X_test)
print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96       198
         1.0       0.00      0.00      0.00        18

    accuracy                           0.92       216
   macro avg       0.46      0.50      0.48       216
weighted avg       0.84      0.92      0.88       216



/home/e/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
#second Outlier Detection
clf=svm.OneClassSVM(nu=.2,kernel='rbf',gamma=.001)
clf.fit(df2)
y_pred=clf.predict(df2)

In [20]:
y_pred.shape


(800,)

In [19]:
TT=df2.loc[y_pred==1]
df2=df2.drop(TT.T)
X=df2.iloc[:, :-1].values
y=df2.iloc[:, 6060].values

IndexError: Item wrong length 800 instead of 160.

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [15]:
svm2_clf = svm.SVC(kernel='linear')

from sklearn import svm, metrics

svm2_clf.fit(X_train, y_train)
expected = y_test
predicted = svm2_clf.predict(X_test)
print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

          -1       0.86      0.78      0.82        41
           1       0.18      0.29      0.22         7

    accuracy                           0.71        48
   macro avg       0.52      0.53      0.52        48
weighted avg       0.77      0.71      0.73        48

